In [1]:
###get toxicity score from Perspective API
from googleapiclient import discovery
import pandas as pd
import time
import os

In [2]:
def get_toxicity_score(comment):
    # print(comment)
    analyze_request = {
      'comment': { 'text': comment},
    #  'comment': { 'text': 'friendly greetings from python'},
      'requestedAttributes': {'TOXICITY': {}}
    }

    response = service.comments().analyze(body=analyze_request).execute()
#     import json
#     print(json.dumps(response, indent=2))
    toxicity_score = response['attributeScores']['TOXICITY']['summaryScore']['value']
    return toxicity_score

###main()

# Generates Perspective API client object dynamically based on service name and version.
API_KEY = pd.read_csv('perspective-api-key.txt', names = ['key'])['key'][0]
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)

comment = "Wtf bruh?"

###per comment
try:
    toxicity_score = get_toxicity_score(comment)
except:
    toxicity_score = -1.0

print("Toxicity score from Perspective API = ", toxicity_score)    

Toxicity score from Perspective API =  0.6191686


# Get toxicity scores for r/TRP comments

In [7]:
filepath = "/mnt/storage/quarantine/data/TheRedPill-comments.csv"

tf = pd.read_csv(filepath)

/nethome/eshwar3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
# # tf = tf[(tf.body != '[deleted]') & (tf.body != '[removed]')]
# count = 0
# toxicity_scores = []
# limit = 10

# for comment in tf[:limit].body:
#     count += 1
#     print(count)
#     if ((comment == "[deleted]") | (comment == "[removed]")):
#         score = -1.0
#         toxicity_scores.append(score)

#     else:
#         try:
#             score = get_toxicity_score(comment)
#             toxicity_scores.append(score)
#             time.sleep(1)      ###rate limit: 1 request per second
#         except:
#             score = -1.0
#             toxicity_scores.append(score)

# tf['toxicity'] = toxicity_scores
# # tf.to_csv("/mnt/storage/quarantine/data/theRedPillComments_toxicity.csv", index = False)

1
2
3
4
5


In [8]:
tf.shape

(235553, 20)

# Toxicity score computation using multiple keys

In [9]:
###split the dataframe into <num_keys> smaller dataframes
filepath = "/mnt/storage/quarantine/data/TheRedPill-comments.csv"
tf = pd.read_csv(filepath)

df = tf
kf = pd.read_csv("perspective-api-keys.txt", names = ['key'])
num_keys = kf.shape[0]
print(num_keys)

chunk_size = int(df.shape[0]/num_keys)
print(chunk_size)

curr = 0
df_list = []

for i in range(num_keys):
    if i < num_keys - 1:
        print(i, curr, curr+chunk_size)
        df_list.append(df[curr:curr+chunk_size])
        curr += chunk_size
    else:
        df_list.append(df[curr:])

11
21413
0 0 21413
1 21413 42826
2 42826 64239
3 64239 85652
4 85652 107065
5 107065 128478
6 128478 149891
7 149891 171304
8 171304 192717
9 192717 214130


In [111]:
###create multiple subfiles
count = 0
for ff in df_list:
    ff.to_csv('subdf-' + str(count) + '.csv', index = False)
    count += 1

In [10]:
###score all subfiles individually

In [ ]:
def score_comments(key_id):
    chunk_num = key_id
    ###get toxicity score from Perspective API
    from googleapiclient import discovery
    import pandas as pd
    import time

    def get_toxicity_score(comment):
        # print(comment)
        analyze_request = {
          'comment': { 'text': comment},
        #  'comment': { 'text': 'friendly greetings from python'},
          'requestedAttributes': {'TOXICITY': {}}
        }

        response = service.comments().analyze(body=analyze_request).execute()
        toxicity_score = response['attributeScores']['TOXICITY']['summaryScore']['value']
        return toxicity_score

    ###main()
    # Generates Perspective API client object dynamically based on service name and version.
    API_KEY = pd.read_csv('perspective-api-keys.txt.txt', names = ['key'])['key'][chunk_num]
    service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)
    filename = "subdf-" + str(chunk_num)

    # tf = tf[(tf.body != '[deleted]') & (tf.body != '[removed]')]
    count = 0
    toxicity_scores = []

    tf = pd.read_csv(filename + ".csv")
    tf = tf[:limit]
    print(chunk_num, tf.shape[0])

    for comment in tf.body:
        print(count)
        count += 1
        if ((comment == "[deleted]") | (comment == "[removed]")):
            score = -1.0
            toxicity_scores.append(score)
        else:
            try:
                score = get_toxicity_score(comment)
                toxicity_scores.append(score)
                time.sleep(1)      ###rate limit: 1 request per second
            except:
                score = -1.0
                toxicity_scores.append(score)

    tf['toxicity'] = toxicity_scores
    tf.to_csv(filename + "-toxicity.csv", index = False)

In [11]:
###merge the <num_keys> smaller dataframes into a single dataframe
for i in range(num_keys):
    print(i)
    temp = pd.read_csv('subdf-' + str(i) + '-toxicity.csv')
    if i == 0:
        final = temp
    else:
        final = pd.concat([final, temp])

final.to_csv("trp-comments-toxicity-scores.csv", index = False)

0
1
2
3
4
5
6
7
8
9
10


# parallel-scoring

In [2]:
from multiprocessing import Process

def main(): 
    def test_func(x):
        for i in range(15):
            print(x)
        print("Done! ", x)

    proc=[]
    param_list = ["eshwar", "server", "automod", "lbj"]
    for num in range(len(param_list)):
        p = Process(target=test_func,args=(param_list[num],))
        proc.append(p)

    # Attempting to run simultaneously. This is where the error occurs:
    for p in proc:
        p.start()

    # Join after all processes started, so you actually get parallelism
    for p in proc:
        p.join()

if __name__ == '__main__':
    main()

eshwar
server
eshwar
server
eshwar
eshwar
server
eshwar
server
eshwar
automod
server
eshwar
automod
server
eshwar
automod
server
automod
eshwar
automod
server
lbj
eshwar
automod
server
eshwar
lbj
automod
server
automod
eshwar
lbj
server
eshwar
automod
lbj
server
eshwar
automod
lbj
eshwar
automod
server
lbj
Done!  eshwar
automod
server
lbj
automod
server
automod
lbj
Done!  server
automod
lbj
Done!  automod
lbj
lbj
lbj
lbj
lbj
lbj
Done!  lbj


# Parallel scoring of comments using multiple Toxicity API keys

In [ ]:
from multiprocessing import Process
import pandas as pd

def main(): 
    ###split the dataframe into <num_keys> smaller dataframes
    data_path = "/mnt/storage/quarantine/data/TheRedPill-comments.csv"
    keys_path = "perspective-api-keys.txt"
    
    tf = pd.read_csv(data_path)
    df = tf
    kf = pd.read_csv(keys_path, names = ['key'])
    num_keys = kf.shape[0]
    print("Number of keys available = ", num_keys)

    chunk_size = int(df.shape[0]/num_keys)
    print("Size of sub_df chunks = ", chunk_size)

    curr = 0
    df_list = []

    for i in range(num_keys):
        if i < num_keys - 1:
            print(i, curr, curr+chunk_size)
            df_list.append(df[curr:curr+chunk_size])
            curr += chunk_size
        else:
            df_list.append(df[curr:])

    ###helper function that'll be used for multi-processing
    def score_comments(key_id):
        
        chunk_num = key_id
        ###get toxicity score from Perspective API
        from googleapiclient import discovery
        import pandas as pd
        import time

        def get_toxicity_score(comment):
            # print(comment)
            analyze_request = {
              'comment': { 'text': comment},
            #  'comment': { 'text': 'friendly greetings from python'},
              'requestedAttributes': {'TOXICITY': {}}
            }

            response = service.comments().analyze(body=analyze_request).execute()
            toxicity_score = response['attributeScores']['TOXICITY']['summaryScore']['value']
            return toxicity_score

        # Generates Perspective API client object dynamically based on service name and version.
        API_KEY = pd.read_csv('perspective-api-keys.txt.txt', names = ['key'])['key'][chunk_num]
        service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=API_KEY)
#         filename = "subdf-" + str(chunk_num)
#         tf = pd.read_csv(filename + ".csv")
        tf = df_list[chunk_num]

        count = 0
        toxicity_scores = []

        print(chunk_num, tf.shape[0])

        for comment in tf.body:
            print(count)
            count += 1
            if ((comment == "[deleted]") | (comment == "[removed]")):
                score = -1.0
                toxicity_scores.append(score)
            else:
                try:
                    score = get_toxicity_score(comment)
                    toxicity_scores.append(score)
                    time.sleep(1)      ###rate limit: 1 request per second
                except:
                    score = -1.0
                    toxicity_scores.append(score)

        tf['toxicity'] = toxicity_scores
        tf.to_csv("chunk-" + str(chunk_num) + "-toxicity.csv", index = False)

    ####create the parallel processes
    proc=[]
    for num in range(num_keys):
        p = Process(target=test_func,args=(num,))
        proc.append(p)

    # Attempting to run simultaneously. This is where the error occurs:
    for p in proc:
        p.start()

    # Join after all processes started, so you actually get parallelism
    for p in proc:
        p.join()

if __name__ == '__main__':
    main()

# Raw-

In [18]:
data = pd.read_csv("/mnt/storage/quarantine/data/TheRedPill-comments-toxicity.csv")

/nethome/eshwar3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [28]:
data[data.toxicity > 0.8]

,body,score_hidden,archived,name,author,author_flair_text,downs,created_utc,subreddit_id,link_id,parent_id,score,retrieved_on,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,toxicity
2,Sometimes I wonder what I'm even doing here wh...,False,False,NaN,FrontloadPain,NaN,NaN,1522594628,t5_2ve1u,t3_887fk2,t1_dwm23d7,-4,1525820846,0,0,dwmng5v,TheRedPill,NaN,NaN,NaN,0.802834
3,George Clooney is not a fucking leader LMAO \n...,False,False,NaN,bby_gem,NaN,NaN,1522552861,t5_2ve1u,t3_88bja6,t1_dwkpclc,-8,1525808064,0,0,dwm0qrp,TheRedPill,NaN,NaN,NaN,0.871854
4,"Don’t do that, that’s gay dude",False,False,NaN,scissor_me_timbers00,1,NaN,1522558744,t5_2ve1u,t3_88l5h5,t1_dwlwjw9,-12,1525810396,0,0,dwm501a,TheRedPill,NaN,NaN,points,0.901688
24,&gt;If you spent 5 hours a week reading about ...,False,False,NaN,BobbyPeru,NaN,NaN,1522555922,t5_2ve1u,t3_88o247,t3_88o247,338,1525809319,0,0,dwm3274,TheRedPill,NaN,NaN,NaN,0.837852
31,"Depends, if you're a short guy deadlifting wil...",False,False,NaN,JaxHerer,NaN,NaN,1522574927,t5_2ve1u,t3_88owzj,t1_dwmclmw,-5,1525814979,0,0,dwmcsua,TheRedPill,NaN,NaN,NaN,0.897008
38,Right on! I had a similar situation not long a...,False,False,NaN,PhaedrusHunt,NaN,NaN,1522610680,t5_2ve1u,t3_88p58n,t3_88p58n,23,1525828396,0,0,dwn1zf7,TheRedPill,NaN,NaN,NaN,0.837852
45,"Thats what sucks about being 5'7, you really h...",False,False,NaN,Solitary_Solidarity,NaN,NaN,1522598299,t5_2ve1u,t3_88pl6c,t3_88pl6c,21,1525822527,0,0,dwmqmnw,TheRedPill,NaN,NaN,NaN,0.949595
52,"Probably a foreigner, those fuck heads literal...",False,False,NaN,MikeTheJew,NaN,NaN,1522606825,t5_2ve1u,t3_88pl6c,t1_dwmqv32,74,1525826603,0,0,dwmyhsr,TheRedPill,NaN,NaN,NaN,0.905621
58,This is nothing but bullshit. You just want va...,False,False,NaN,chunderous,NaN,NaN,1522586680,t5_2ve1u,t3_88prhg,t3_88prhg,65,1525817789,0,0,dwmhush,TheRedPill,NaN,NaN,NaN,0.837852
64,It's only beta if you don't look him in the ey...,False,False,NaN,max_peenor,Senior Endorsed Contributor,NaN,1522686791,t5_2ve1u,t3_88pud4,t1_dwmezhs,39,1525868492,0,0,dwoifyp,TheRedPill,NaN,NaN,endorsementsenior points,0.863223


# Double-check that comments with toxicity = -1.0 were scored by API 

In [29]:
validation_df = data[(data.toxicity == -1) & (data.body != "[removed]") & (data.body != "[deleted]")]

In [33]:
validation_df.shape, data.shape, data[data.toxicity == -1].shape, data[data.body.isin(["[removed]", "[deleted]"])].shape

((528, 21), (235553, 21), (41585, 21), (41057, 21))

In [71]:
data[(data.toxicity == -1)].shape[0] - data[(data.body == "[removed]") | (data.body == "[deleted]")].shape[0]

447

In [49]:
validation_scores = []
hits = 0

for comment in validation_df.body:
    try:
        score = get_toxicity_score(comment)
        hits += 1
        print(score, comment)
        validation_scores.append(score)
    except Exception as E:
#         print(E)
        validation_scores.append(-1)

print("# hits = ", hits)

0.061961103 mistake 1 you told her that you wanted to see her more (asking for a relationship in a vague way) never bring up being exclusive 

mistake 2 you initiated texting then double texted when she didn’t respond (never double text. if she doesn’t respond to text then so be it)

do not reach back to her. you can’t make her love you


0.6312218 My anecdote is that I've been with many women who are not taking any hormonal contraceptive and then start taking it after we start banging she goes onto it and it completely changes her behavior.  I dread it.  It goes from "this is great" to "this sucks" real fast.
0.21968964 yeah, look at all the great men in history- mostly unmarried. plus, the few that were married were totally disrespected by their wives.../s

marraige isn’t a lost cause and anyone reading this should take this as a reminder that this sub gives bad advice a lot
0.07631419 theres only 2 robocop movies
0.36257204 I'm not labelling anyone a Nazi.

I'm pointing out that dis

0.7874193 Really? 

I think both sides of argument have their own rights and both are wrong at the same time. 

Grasshoppers: "have fun now, because who knows what will come"... I don't see anything wrong about it. That Aesops tale showed the grasshopper as an idiot, because we all know that the winter eventually comes... really? People who live closer to the equator perceive time differently. Perhaps the winter is the reason why whites had no problem with colonising the world. So, in the tale grasshopper is an idiot. How big are your chances of cancer? Your DNA plays an important role, environment plays another. Do you plan to die in an accident? Are you going to loose a limb only or do you plan for something more serious?

Watching chronically ill people at a few institutions shows clearly, that when you're fucked - you're fucked. A few stories from fellow TRPers also seem to back this up. 

Ants save. Can you loose when you save? Doesn't look like it... really? 

I suddenly realised

0.84733623 How old are you? How many women have you fucked? 

How is the social status of having a girlfriend going to improve your life?

Why do you want to be in a relationship? Are you lonely? You want to seem cool and peer-approved by having some arm candy? 

Post this to asktrp, but be ready to be ripped a new asshole because generally the type of guys who WANT to be in a relationship are 100&amp;#37; the guys who really shouldn't be in one. 
0.7625084 TRP fanfiction...let is slide.....   

A high value extraordinary man that was truly the mega-Chad behemoth of pussy slaying that had a swimsuit model wife cooking him beans and franks while he was out getting strange pussy smell on him WOULD NOT BE POSTING about it on reddit and a site about helping socially awkward STEM majors get a girl.   

It is fanfiction to produce a heightened sense of jealousy in the usual suspects so that they can dream about what might happen in the future if they could just understand that last nugget of

0.79996306 The reason men have this view of women is the ease of obtaining sex for women. As a man you have to put in effort to compete(job, look, physique, game, etc). A man has to have something going for himself and be the one to approach and deal with shit tests. So if you can close the deal then you earned it and should be proud.

On the woman’s side they can be fat or just a 5. All they have to do is sit at a bar, or have a social media account and guys will flock to them. They don’t have to put any effort into getting sex. All they have to do is allow a guy to put his dick in her. This is how you have those fat women that want a tall, muscular guy that treats them like a queen and some guy out there will do it. Because some guys have no standards it gives these women high standards.
0.64302033 Yes, you are entitled to do as your wish. But think about this. Being angry is a waste of energy. Arguing, never wins you an argument, it just gets other people's backs up.  Calling a woma

In [51]:
hits

81

In [62]:
###update toxicity for the 81 comments that appear to have been missed in the first iteration of scoring
hits = 0

for i, row in data.iterrows():
    if (row['toxicity'] == -1) & (row['body']!="[removed]") & (row['body']!="[deleted]"):
        print("Validating: ", i)
        try:
            score = get_toxicity_score(row['body'])
            print(score, comment)
            hits += 1
            data.at[i, 'toxicity'] = score
        except Exception as E:
#             print(E)
            continue

# df.at[i,'ifor'] = ifor_val
print(hits)

Validating:  120
Validating:  605
Validating:  1316
Validating:  1324
Validating:  1552
Validating:  1670
Validating:  2182
Validating:  3349
Validating:  3440
Validating:  3774
Validating:  4147
Validating:  4207
0.061961103 https://knowyourmeme.com/memes/chad-thundercock
Validating:  4231
0.6312218 https://knowyourmeme.com/memes/chad-thundercock
Validating:  6301
Validating:  6418
0.21968964 https://knowyourmeme.com/memes/chad-thundercock
Validating:  10011
Validating:  10967
Validating:  11042
Validating:  12099
Validating:  14492
Validating:  15048
Validating:  15912
Validating:  17227
Validating:  17741
Validating:  17992
Validating:  19532
Validating:  20338
Validating:  20978
Validating:  24101
Validating:  24255
Validating:  24368
Validating:  24741
Validating:  25394
Validating:  25551
Validating:  26109
Validating:  26694
Validating:  26851
Validating:  27279
Validating:  27996
Validating:  28268
Validating:  28486
Validating:  28666
Validating:  28880
Validating:  29053
Vali

Validating:  139607
Validating:  139615
Validating:  139651
0.6110324 https://knowyourmeme.com/memes/chad-thundercock
Validating:  139782
Validating:  140137
Validating:  140182
Validating:  140810
Validating:  140856
0.097935334 https://knowyourmeme.com/memes/chad-thundercock
Validating:  140998
Validating:  141177
0.161626 https://knowyourmeme.com/memes/chad-thundercock
Validating:  141372
Validating:  141403
0.55838543 https://knowyourmeme.com/memes/chad-thundercock
Validating:  142774
Validating:  143071
Validating:  143252
Validating:  143655
Validating:  144036
Validating:  144047
Validating:  144063
Validating:  144464
Validating:  144823
Validating:  145140
Validating:  145736
Validating:  146614
Validating:  147158
Validating:  148658
Validating:  148705
Validating:  149073
Validating:  149074
Validating:  149108
Validating:  149177
Validating:  149448
Validating:  150210
Validating:  150929
Validating:  151419
Validating:  151657
Validating:  151659
Validating:  151766
Valida

In [70]:
data.to_csv("/mnt/storage/quarantine/data/theRedPillComments-toxicity.csv", index = False)

In [75]:
data[data.toxicity != -1].shape

(194049, 21)

# Get toxicity scores for user timelines

In [82]:
filepath = "/mnt/storage/quarantine/data/redpill_users_history/"
total = 0

for f in os.listdir(filepath):
    timeline_df = pd.read_csv(filepath + f)
    temp = timeline_df.shape[0]
    print(f, temp)
    total += temp
    
print("Total comments to score = ", total)

redpill-user-history-000000000068.csv 205341
redpill-user-history-000000000037.csv 205360
redpill-user-history-000000000085.csv 206246
redpill-user-history-000000000009.csv 205822
redpill-user-history-000000000088.csv 205593
redpill-user-history-000000000093.csv 205796
redpill-user-history-000000000091.csv 204833
redpill-user-history-000000000067.csv 205684
redpill-user-history-000000000078.csv 205746
redpill-user-history-000000000053.csv 206443
redpill-user-history-000000000028.csv 205038
redpill-user-history-000000000029.csv 205283
redpill-user-history-000000000021.csv 205970
redpill-user-history-000000000073.csv 205557
redpill-user-history-000000000090.csv 206138
redpill-user-history-000000000082.csv 206720
redpill-user-history-000000000059.csv 205492
redpill-user-history-000000000095.csv 205299
redpill-user-history-000000000072.csv 206188
redpill-user-history-000000000027.csv 205606
redpill-user-history-000000000098.csv 205097
redpill-user-history-000000000063.csv 204594
redpill-us

/nethome/eshwar3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


redpill-user-history-000000000069.csv 205115
redpill-user-history-000000000026.csv 205311
redpill-user-history-000000000041.csv 205334
redpill-user-history-000000000038.csv 206026
redpill-user-history-000000000010.csv 205927
redpill-user-history-000000000048.csv 205772
redpill-user-history-000000000012.csv 205407
redpill-user-history-000000000032.csv 205103
redpill-user-history-000000000057.csv 206801
Total comments to score =  20558941


# Batch processing to get toxicity scores for new data samples
July 29, 2019

In [13]:
#score comments

In [1]:
import requests
from googleapiclient import discovery
import pandas as pd
import time

RED_PILL_COMMENTS_FILE = "/mnt/storage/quarantine/data/TheRedPill-comments.csv"
comments_data = pd.read_csv(RED_PILL_COMMENTS_FILE)

comment_list = comments_data['body']
print(len(comment_list))

KEY = pd.read_csv("whitelist-perspective-api-key.txt", names = ['key'])['key'][0]
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=KEY)

def get_results(request_id, response, exception):
    toxicity_scores.append((request_id, response))

count = 0
limit = 499
toxicity_scores = []
batch = service.new_batch_http_request(callback=get_results)

iteration = 0

'''
TOXICITY: rude, disrespectful, or unreasonable comment that is likely to make people leave a discussion.
SEVERE_TOXICITY: a very hateful, aggressive, disrespectful comment or otherwise very likely to make a user leave a discussion or give up on sharing their perspective. This model is much less sensitive to comments that include positive uses of curse words, for example. A labelled dataset and details of the methodolgy can be found in the same toxicity dataset that is available for the toxicity model.
IDENTITY_ATTACK: negative or hateful comments targeting someone because of their identity.
INSULT: insulting, inflammatory, or negative comment towards a person or a group of people.
PROFANITY: swear words, curse words, or other obscene or profane language.
THREAT: describes an intention to inflict pain, injury, or violence against an individual or group.
SEXUALLY_EXPLICIT: contains references to sexual acts, body parts, or other lewd content.
FLIRTATION: pickup lines, complimenting appearance, subtle sexual innuendos, etc.
###
ATTACK_ON_AUTHOR: Attack on the author of an article or post.
ATTACK_ON_COMMENTER: Attack on fellow commenter.
INCOHERENT: Difficult to understand, nonsensical.
INFLAMMATORY: Intending to provoke or inflame.
LIKELY_TO_REJECT: Overall measure of the likelihood for the comment to be rejected according to the NYT's moderation.
OBSCENE: Obscene or vulgar language such as cursing.
SPAM: Irrelevant and unsolicited commercial content.
UNSUBSTANTIAL: Trivial or short comments.
'''

###list of the different types of toxicity scores that are to be included in the dataset
list_toxicity_models = [
                        "TOXICITY",
                        "SEVERE_TOXICITY",
                        "IDENTITY_ATTACK",
                        "INSULT",
                        "PROFANITY",
                        "THREAT",
                        "SEXUALLY_EXPLICIT",
                        "FLIRTATION",
                        "ATTACK_ON_COMMENTER",
                        "INFLAMMATORY",
                        "OBSCENE"
                    ]

for comment in comment_list:
    analyze_request = {
      'comment': { 'text': comment},
      "requestedAttributes": {
        "TOXICITY": {},
        "SEVERE_TOXICITY": {},
        "IDENTITY_ATTACK": {},
        "INSULT": {},
        "PROFANITY": {},
        "THREAT": {},
        "SEXUALLY_EXPLICIT" : {},
        "FLIRTATION": {},
        "ATTACK_ON_AUTHOR": {},
        "ATTACK_ON_COMMENTER": {},
        "INCOHERENT": {},
        "INFLAMMATORY": {},
        "LIKELY_TO_REJECT": {},
        "OBSCENE": {},
        "SPAM": {},
        "UNSUBSTANTIAL": {}
      }
    }
    count += 1
    
    batch.add(service.comments().analyze(body=analyze_request), request_id=str(count))
        
    if count >= limit:
        batch.execute()
        batch = service.new_batch_http_request(callback=get_results)
        count = 0
        print("Sleep #", iteration)
        iteration += 1
        time.sleep(2)
#         break

batch.execute()
print("Done")

/nethome/eshwar3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


232859
Sleep # 0
Sleep # 1
Sleep # 2
Sleep # 3
Sleep # 4
Sleep # 5
Sleep # 6
Sleep # 7
Sleep # 8
Sleep # 9
Sleep # 10
Done


In [5]:
toxicity = {}
for toxicity_model in list_toxicity_models:
    toxicity[toxicity_model] = []

misses = 0
for i in range(len(comment_list)):
    try:
        for toxicity_model in list_toxicity_models:
            toxicity[toxicity_model].append(toxicity_scores[i][1]['attributeScores'][toxicity_model]['summaryScore']['value'])
    except:
#         print(i, "Messed up!")
        misses += 1:wq
        
        for toxicity_model in list_toxicity_models:
            toxicity[toxicity_model].append(-1.0)

print(len(comment_list), len(toxicity))

for toxicity_model in list_toxicity_models:
    comments_data[toxicity_model] = toxicity[toxicity_model]

comments_data.to_csv('/mnt/storage/quarantine/data/TheRedPill-comments-toxicity-scores.csv', index = False)
print("Written to file!")

232859 11
Written to file!


In [ ]:
comments_data.mean()

In [12]:
#score submissions

In [17]:
import requests
from googleapiclient import discovery
import pandas as pd
import time

#score comments
RED_PILL_SUBMISSIONS_FILE = "/mnt/storage/quarantine/data/TheRedPill-submissions.csv"
submissions_data = pd.read_csv(RED_PILL_SUBMISSIONS_FILE)

submission_list = submissions_data['title']
print(len(submission_list))

KEY = pd.read_csv("whitelist-perspective-api-key.txt", names = ['key'])['key'][0]
service = discovery.build('commentanalyzer', 'v1alpha1', developerKey=KEY)

def get_results(request_id, response, exception):
    toxicity_scores.append((request_id, response))

count = 0
limit = 498
toxicity_scores = []
batch = service.new_batch_http_request(callback=get_results)

iteration = 0

'''
TOXICITY: rude, disrespectful, or unreasonable comment that is likely to make people leave a discussion.
SEVERE_TOXICITY: a very hateful, aggressive, disrespectful comment or otherwise very likely to make a user leave a discussion or give up on sharing their perspective. This model is much less sensitive to comments that include positive uses of curse words, for example. A labelled dataset and details of the methodolgy can be found in the same toxicity dataset that is available for the toxicity model.
IDENTITY_ATTACK: negative or hateful comments targeting someone because of their identity.
INSULT: insulting, inflammatory, or negative comment towards a person or a group of people.
PROFANITY: swear words, curse words, or other obscene or profane language.
THREAT: describes an intention to inflict pain, injury, or violence against an individual or group.
SEXUALLY_EXPLICIT: contains references to sexual acts, body parts, or other lewd content.
FLIRTATION: pickup lines, complimenting appearance, subtle sexual innuendos, etc.
###
ATTACK_ON_AUTHOR: Attack on the author of an article or post.
ATTACK_ON_COMMENTER: Attack on fellow commenter.
INCOHERENT: Difficult to understand, nonsensical.
INFLAMMATORY: Intending to provoke or inflame.
LIKELY_TO_REJECT: Overall measure of the likelihood for the comment to be rejected according to the NYT's moderation.
OBSCENE: Obscene or vulgar language such as cursing.
SPAM: Irrelevant and unsolicited commercial content.
UNSUBSTANTIAL: Trivial or short comments.
'''

###list of the different types of toxicity scores that are to be included in the dataset
list_toxicity_models = [
                        "TOXICITY",
                        "SEVERE_TOXICITY",
                        "IDENTITY_ATTACK",
                        "INSULT",
                        "PROFANITY",
                        "THREAT",
                        "SEXUALLY_EXPLICIT",
                        "FLIRTATION",
                        "ATTACK_ON_COMMENTER",
                        "INFLAMMATORY",
                        "OBSCENE"
                    ]

for submission in submission_list:
    analyze_request = {
      'comment': { 'text': submission},
      "requestedAttributes": {
        "TOXICITY": {},
        "SEVERE_TOXICITY": {},
        "IDENTITY_ATTACK": {},
        "INSULT": {},
        "PROFANITY": {},
        "THREAT": {},
        "SEXUALLY_EXPLICIT" : {},
        "FLIRTATION": {},
        "ATTACK_ON_AUTHOR": {},
        "ATTACK_ON_COMMENTER": {},
        "INCOHERENT": {},
        "INFLAMMATORY": {},
        "LIKELY_TO_REJECT": {},
        "OBSCENE": {},
        "SPAM": {},
        "UNSUBSTANTIAL": {}
      }
    }
    
    count += 1    
    batch.add(service.comments().analyze(body=analyze_request), request_id=str(count))
    
    if count >= limit:
        batch.execute()
        batch = service.new_batch_http_request(callback=get_results)
        count = 0
        print("Sleep #", iteration)
        iteration += 1
        time.sleep(2)
#         break

batch.execute()
print("Scoring = Done")

10899
Sleep # 0
Sleep # 1
Sleep # 2
Sleep # 3
Sleep # 4
Sleep # 5
Sleep # 6
Sleep # 7
Sleep # 8
Sleep # 9
Sleep # 10
Sleep # 11
Sleep # 12
Sleep # 13
Sleep # 14
Sleep # 15
Sleep # 16
Sleep # 17
Sleep # 18
Sleep # 19
Sleep # 20
Done


In [18]:
toxicity = {}
for toxicity_model in list_toxicity_models:
    toxicity[toxicity_model] = []

misses = 0
for i in range(len(submission_list)):
    try:
        for toxicity_model in list_toxicity_models:
            toxicity[toxicity_model].append(toxicity_scores[i][1]['attributeScores'][toxicity_model]['summaryScore']['value'])
    except:
#         print(i, "Messed up!")
        misses += 1
        
        for toxicity_model in list_toxicity_models:
            toxicity[toxicity_model].append(-1.0)

print(len(submission_list), len(toxicity))

for toxicity_model in list_toxicity_models:
    submissions_data[toxicity_model] = toxicity[toxicity_model]

submissions_data.to_csv('/mnt/storage/quarantine/data/TheRedPill-submissions-toxicity-scores.csv', index = False)
print("Written to file!")

10899 10899
Written to file!


In [19]:
1

1